## 1. 📦 Import Libraries

Import all necessary libraries for data processing, deep learning, and anomaly detection.

In [ ]:
# Standard libraries
import os
import random
import time
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm.auto import tqdm
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Tuple

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

# FAISS with safe fallback to CPU if GPU module missing
try:
    import faiss
except ImportError:
    print("Installing faiss-gpu...")
    os.system("pip uninstall -y faiss-cpu")  # Remove CPU version to prevent conflicts
    os.system("pip install faiss-gpu -q")
    import faiss

print("✓ All libraries imported successfully")

## 2. ⚙️ Configuration & Setup

Configure hyperparameters, paths, and environment setup.

In [ ]:
# =============================================================================
# MODEL & EXTRACTION HYPERPARAMETERS
# =============================================================================

IMG_SIZE = 224              # I3D input size
SEQ_LEN = 16                # Frames per clip (I3D requirement)
BATCH_SIZE = 4              # Batch size for I3D extraction
K_NEIGHBORS = 5             # Number of neighbors for KNN
NUM_WORKERS = 4             # Parallel workers for image loading

# =============================================================================
# CACHE & STORAGE CONFIGURATION
# =============================================================================

CACHE_DIR = "/kaggle/working/cache"
FEATURE_CACHE = os.path.join(CACHE_DIR, "features")

# =============================================================================
# DATASET PATHS
# =============================================================================

DATA_ROOT = "/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "training_videos")
TEST_DIR = os.path.join(DATA_ROOT, "testing_videos")
OUTPUT_CSV = "avenue_scores.csv"

# =============================================================================
# REPRODUCIBILITY & DEVICE
# =============================================================================

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create cache directory
os.makedirs(FEATURE_CACHE, exist_ok=True)

print(f"✓ Device: {device}")
print(f"✓ Cache: {FEATURE_CACHE}")

## 3. 🖼️ Vectorized Image Loading

Efficient parallel image loading with vectorized normalization using ImageNet statistics.

In [ ]:
# =============================================================================
# NORMALIZATION CONSTANTS (ImageNet)
# =============================================================================

MEAN = np.array([0.45, 0.45, 0.45], dtype=np.float32).reshape(1, 1, 1, 3)
STD = np.array([0.225, 0.225, 0.225], dtype=np.float32).reshape(1, 1, 1, 3)


def load_single_image_raw(path: str) -> np.ndarray:
    """
    Load a single image without normalization (uint8).
    
    Args:
        path: Path to image file
        
    Returns:
        img: (H, W, C) uint8 numpy array
    """
    img = cv2.imread(path)
    if img is None:
        return np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    # Convert BGR to RGB and resize
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
    return img


def load_images_vectorized(paths: List[str]) -> np.ndarray:
    """
    VECTORIZED: Load multiple images in parallel with efficient normalization.
    
    Uses ThreadPoolExecutor for parallel I/O and numpy broadcasting for
    fast vectorized normalization.
    
    Args:
        paths: List of image file paths
        
    Returns:
        batch: (N, H, W, C) float32 normalized numpy array
    """
    # Parallel I/O loading
    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        images = list(executor.map(load_single_image_raw, paths))
    
    # VECTORIZED: Stack and normalize in one operation
    batch = np.stack(images, axis=0).astype(np.float32)  # (N, H, W, C)
    batch = (batch / 255.0 - MEAN) / STD  # Vectorized normalization
    
    return batch


def load_clip_vectorized(frame_paths: List[str]) -> np.ndarray:
    """
    Load a video clip with vectorized normalization.
    
    Args:
        frame_paths: List of frame image paths
        
    Returns:
        clip: (T, H, W, C) float32 normalized numpy array
    """
    return load_images_vectorized(frame_paths)


def get_frame_num(path: str) -> int:
    """
    Extract frame number from filename.
    
    Args:
        path: Path to frame image
        
    Returns:
        frame_num: Integer frame number
    """
    name = os.path.basename(path)
    digits = ''.join(filter(str.isdigit, os.path.splitext(name)[0]))
    return int(digits) if digits else 0


print("✓ Image loading functions defined")

## 4. 🧠 I3D Feature Extractor

I3D (Inflated 3D ConvNet) extracts 2048-dimensional features from video clips using a pre-trained ResNet-50 backbone.

In [ ]:
class I3D(nn.Module):
    """
    Singleton I3D model wrapper (loads only once).
    
    Uses Facebook's PyTorchVideo i3d_r50 (ResNet-50 backbone).
    Pre-trained on Kinetics dataset for action recognition.
    
    Architecture:
        Input:  (B, C=3, T=16, H=224, W=224)
        Output: (B, 2048) feature vectors
    """
    _instance = None  # Singleton instance
    
    def __new__(cls):
        # Return existing instance if available
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self):
        # Skip re-initialization if already done
        if self._initialized:
            return
        super().__init__()
        
        print("Loading I3D model from PyTorchVideo...")
        
        # Load pretrained I3D ResNet-50
        self.model = torch.hub.load(
            'facebookresearch/pytorchvideo',
            'i3d_r50',
            pretrained=True
        )
        
        # Remove classification head, keep features only
        self.model.blocks[-1].proj = nn.Identity()
        
        # Freeze all parameters (inference only)
        for p in self.parameters():
            p.requires_grad = False
        
        self.eval()
        self._initialized = True
        print("✓ I3D model loaded")
    
    @torch.no_grad()
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Extract features from video clips.
        
        Args:
            x: Input tensor of shape (B, C, T, H, W)
            
        Returns:
            features: Output tensor of shape (B, 2048)
        """
        return self.model(x)


print("✓ I3D class defined")

## 5. 📊 Feature Extraction Functions

Extract and normalize I3D features from training and test videos with caching for faster re-runs.

In [ ]:
def extract_features_vectorized(data_dir: str, name: str, stride: int = 1) -> Tuple:
    """
    VECTORIZED feature extraction with caching and normalization.
    
    Steps:
        1. Create video clips from frames
        2. Extract I3D features for each clip
        3. Compute normalization statistics (mean, std)
        4. Normalize all features
        5. Cache results to disk
    
    Args:
        data_dir: Directory containing video folders
        name: Name for caching (e.g., "train")
        stride: Stride between consecutive clips
        
    Returns:
        features: (N, 2048) normalized tensor
        clips: List of clip metadata
        videos: Dict of video metadata
        stats: Normalization statistics {mean, std}
    """
    cache_file = os.path.join(FEATURE_CACHE, f"{name}_v2_s{stride}.pt")
    
    # Check cache first
    if os.path.exists(cache_file):
        print(f"[CACHE HIT] Loading {name}")
        data = torch.load(cache_file, weights_only=False)
        return data['features'], data['clips'], data['videos'], data['stats']
    
    print(f"\nExtracting features: {name} (stride={stride})")
    i3d = I3D().to(device)
    
    # Collect video information
    videos = {}
    clips = []
    
    video_dirs = sorted([d for d in os.listdir(data_dir)
                        if os.path.isdir(os.path.join(data_dir, d))])
    print(f"Found {len(video_dirs)} videos")
    
    # Process each video
    for vid_id, vid_name in enumerate(video_dirs, 1):
        vid_path = os.path.join(data_dir, vid_name)
        frames = sorted(glob.glob(os.path.join(vid_path, "*.jpg")))
        
        if len(frames) < SEQ_LEN:
            continue
        
        # Extract frame numbers
        frame_nums = np.array([get_frame_num(f) for f in frames])
        
        videos[vid_id] = {
            'name': vid_name,
            'frames': frames,
            'nums': frame_nums,
            'n': len(frames)
        }
        
        # Create clips with specified stride
        n_clips_vid = (len(frames) - SEQ_LEN) // stride + 1
        for i in range(n_clips_vid):
            start = i * stride
            clips.append({
                'vid': vid_id,
                'start': start,
                'paths': frames[start:start + SEQ_LEN],
                'nums': frame_nums[start:start + SEQ_LEN]
            })
    
    n_clips = len(clips)
    print(f"Total clips: {n_clips}")
    
    # VECTORIZED: Pre-allocate feature tensor
    features = torch.zeros(n_clips, 2048, dtype=torch.float32)
    
    # Extract features in batches
    for i in tqdm(range(0, n_clips, BATCH_SIZE), desc="Extracting I3D"):
        batch_clips = clips[i:i + BATCH_SIZE]
        batch_size = len(batch_clips)
        
        # Load all clips for this batch
        batch_data = [load_clip_vectorized(c['paths']) for c in batch_clips]
        batch_array = np.stack(batch_data, axis=0)  # (B, T, H, W, C)
        
        # Convert to PyTorch and permute: (B, T, H, W, C) -> (B, C, T, H, W)
        batch_tensor = torch.from_numpy(batch_array).permute(0, 4, 1, 2, 3).to(device)
        
        # Extract with mixed precision for speed
        with torch.amp.autocast('cuda'):
            feats = i3d(batch_tensor)
        
        features[i:i + batch_size] = feats.cpu()
    
    # VECTORIZED: Compute normalization statistics
    stats = {
        'mean': features.mean(dim=0),       # (2048,)
        'std': features.std(dim=0) + 1e-8   # (2048,) with epsilon
    }
    
    # VECTORIZED: Normalize all features at once
    features = (features - stats['mean'].unsqueeze(0)) / stats['std'].unsqueeze(0)
    
    print(f"Feature stats: mean={features.mean():.4f}, std={features.std():.4f}")
    
    # Save to cache
    torch.save({
        'features': features,
        'clips': clips,
        'videos': videos,
        'stats': stats
    }, cache_file)
    
    # Clean up GPU memory
    del i3d
    torch.cuda.empty_cache()
    
    return features, clips, videos, stats


def extract_test_features_vectorized(data_dir: str, train_stats: dict, stride: int = 1) -> Tuple:
    """
    VECTORIZED test feature extraction using TRAINING normalization stats.
    
    IMPORTANT: Uses training mean/std for normalization to ensure
    consistent feature scaling between train and test!
    
    Args:
        data_dir: Directory containing test video folders
        train_stats: Normalization statistics from training data
        stride: Stride between consecutive clips
        
    Returns:
        features: (N, 2048) normalized tensor
        clips: List of clip metadata
        videos: Dict of video metadata
    """
    cache_file = os.path.join(FEATURE_CACHE, f"test_v2_s{stride}.pt")
    
    # Check cache first
    if os.path.exists(cache_file):
        print(f"[CACHE HIT] Loading test features")
        data = torch.load(cache_file, weights_only=False)
        return data['features'], data['clips'], data['videos']
    
    print(f"Extracting test features (stride={stride})")
    i3d = I3D().to(device)
    
    videos = {}
    clips = []
    
    video_dirs = sorted([d for d in os.listdir(data_dir)
                        if os.path.isdir(os.path.join(data_dir, d))])
    print(f"Found {len(video_dirs)} videos")
    
    # Process each video
    for vid_id, vid_name in enumerate(video_dirs, 1):
        vid_path = os.path.join(data_dir, vid_name)
        frames = sorted(glob.glob(os.path.join(vid_path, "*.jpg")))
        
        if len(frames) < SEQ_LEN:
            continue
        
        frame_nums = np.array([get_frame_num(f) for f in frames])
        
        videos[vid_id] = {
            'name': vid_name,
            'frames': frames,
            'nums': frame_nums,
            'n': len(frames)
        }
        
        # Create clips
        n_clips_vid = (len(frames) - SEQ_LEN) // stride + 1
        for i in range(n_clips_vid):
            start = i * stride
            clips.append({
                'vid': vid_id,
                'start': start,
                'paths': frames[start:start + SEQ_LEN],
                'nums': frame_nums[start:start + SEQ_LEN]
            })
    
    n_clips = len(clips)
    print(f"Total test clips: {n_clips}")
    
    # Pre-allocate feature tensor
    features = torch.zeros(n_clips, 2048, dtype=torch.float32)
    
    # Extract features in batches
    for i in tqdm(range(0, n_clips, BATCH_SIZE), desc="Extracting I3D"):
        batch_clips = clips[i:i + BATCH_SIZE]
        batch_size = len(batch_clips)
        
        batch_data = [load_clip_vectorized(c['paths']) for c in batch_clips]
        batch_array = np.stack(batch_data, axis=0)
        batch_tensor = torch.from_numpy(batch_array).permute(0, 4, 1, 2, 3).to(device)
        
        with torch.amp.autocast('cuda'):
            feats = i3d(batch_tensor)
        
        features[i:i + batch_size] = feats.cpu()
    
    # CRITICAL: Normalize using TRAINING statistics
    features = (features - train_stats['mean'].unsqueeze(0)) / train_stats['std'].unsqueeze(0)
    
    print(f"Feature stats: mean={features.mean():.4f}, std={features.std():.4f}")
    
    # Save to cache
    torch.save({
        'features': features,
        'clips': clips,
        'videos': videos
    }, cache_file)
    
    # Clean up GPU memory
    del i3d
    torch.cuda.empty_cache()
    
    return features, clips, videos


print("✓ Feature extraction functions defined")

## 6. 🔍 FAISS KNN Engine

Efficient nearest neighbor search for anomaly detection using FAISS with safe GPU/CPU fallback.

In [ ]:
def build_faiss_index(features: torch.Tensor):
    """
    Build FAISS index with safe fallback to CPU if GPU module is missing.
    
    FAISS (Facebook AI Similarity Search) provides efficient exact and
    approximate nearest neighbor search.
    
    Args:
        features: (N, 2048) feature tensor
        
    Returns:
        index: FAISS index object (GPU or CPU)
    """
    print("\n" + "="*50)
    print("  Building FAISS Index")
    print("="*50)
    
    # Convert to numpy float32 (FAISS requirement)
    data_np = features.numpy().astype(np.float32)
    d = data_np.shape[1]  # Dimension = 2048
    
    # Create exact L2 search index
    index = faiss.IndexFlatL2(d)
    
    # Try moving to GPU safely
    if torch.cuda.is_available() and hasattr(faiss, 'StandardGpuResources'):
        try:
            print("Moving index to GPU...")
            res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(res, 0, index)
            print("  ✓ GPU Index created")
        except Exception as e:
            print(f"  ⚠ GPU move failed ({e}), using CPU")
    else:
        print("  ℹ Using CPU Index (safe fallback)")
    
    # Add training features to index
    index.add(data_np)
    print(f"Index built with {index.ntotal} vectors")
    
    return index


def compute_knn_scores(index, test_features: torch.Tensor, k: int = 5) -> np.ndarray:
    """
    Compute anomaly scores using KNN distances.
    
    Anomaly Score = Mean L2 distance to k nearest neighbors
    
    Intuition: Normal frames are similar to many training frames (low distance),
    while anomalous frames are dissimilar (high distance).
    
    Args:
        index: FAISS index built from training features
        test_features: (N, 2048) test feature tensor
        k: Number of neighbors to search
        
    Returns:
        scores: (N,) array of anomaly scores
    """
    print(f"\nComputing KNN distances (k={k})...")
    
    # Convert to numpy float32
    test_np = test_features.numpy().astype(np.float32)
    
    # Search: D = distances, I = indices
    D, I = index.search(test_np, k)
    
    # Mean squared distance to k nearest neighbors
    scores = np.mean(D, axis=1)
    
    print(f"Score range: [{scores.min():.4f}, {scores.max():.4f}]")
    
    return scores


print("✓ FAISS KNN functions defined")

## 7. 🚀 Main Pipeline

Complete anomaly detection pipeline integrating all components.

In [ ]:
def run_pipeline():
    """
    Main pipeline: Train -> Build Index -> Test -> Evaluate.
    """
    print("\n" + "="*60)
    print("  AVENUE ANOMALY DETECTION - FAISS KNN")
    print("="*60)
    
    start_time = time.time()
    
    # =================================================================
    # STEP 1: Extract training features
    # =================================================================
    print("\n[1/5] Extracting Training Features")
    train_feats, _, _, stats = extract_features_vectorized(
        TRAIN_DIR, "train", stride=1
    )
    
    # =================================================================
    # STEP 2: Build FAISS index from training features
    # =================================================================
    print("\n[2/5] Building FAISS Index")
    index = build_faiss_index(train_feats)
    
    # =================================================================
    # STEP 3: Extract test features
    # =================================================================
    print("\n[3/5] Extracting Test Features")
    test_feats, clips, videos = extract_test_features_vectorized(
        TEST_DIR, stats, stride=1
    )
    
    # =================================================================
    # STEP 4: Compute anomaly scores
    # =================================================================
    print("\n[4/5] Computing Anomaly Scores")
    clip_scores = compute_knn_scores(index, test_feats, k=K_NEIGHBORS)
    
    # =================================================================
    # STEP 5: Aggregate to frame-level scores
    # =================================================================
    print("\n[5/5] Aggregating Frame Scores")
    video_scores = {}
    frame_score_map = defaultdict(list)
    
    # Map clip scores to frames (center frames are most accurate)
    for i, clip in enumerate(clips):
        vid_id = clip['vid']
        frame_nums = clip['nums']
        score = clip_scores[i]
        
        # Assign to center frames only
        mid_start = SEQ_LEN // 4
        mid_end = SEQ_LEN - SEQ_LEN // 4
        for j in range(mid_start, mid_end):
            frame_score_map[(vid_id, frame_nums[j])].append(score)
    
    # Aggregate scores per video
    all_frame_ids = []
    all_scores = []
    from scipy.ndimage import gaussian_filter1d
    
    for vid_id, info in sorted(videos.items()):
        n_frames = info['n']
        frame_nums = info['nums']
        scores = np.zeros(n_frames, dtype=np.float32)
        
        # Compute max score for each frame (multi-scale detection)
        for i, fn in enumerate(frame_nums):
            key = (vid_id, fn)
            if key in frame_score_map:
                scores[i] = np.max(frame_score_map[key])
        
        # Temporal smoothing with Gaussian filter
        scores = gaussian_filter1d(scores.astype(np.float64), sigma=4)
        video_scores[vid_id] = scores
        
        # Build frame IDs
        all_frame_ids.extend([f"{vid_id}_{fn}" for fn in frame_nums])
        all_scores.extend(scores)
    
    # =================================================================
    # NORMALIZATION & SAVING
    # =================================================================
    
    # Normalize scores using percentiles (robust to outliers)
    all_scores = np.array(all_scores)
    p1, p99 = np.percentile(all_scores, [1, 99])
    all_scores_norm = np.clip(all_scores, p1, p99)
    all_scores_norm = (all_scores_norm - p1) / (p99 - p1 + 1e-8)
    
    # Save results
    df = pd.DataFrame({'Id': all_frame_ids, 'Score': all_scores_norm})
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✓ Saved results to {OUTPUT_CSV}")
    print(f"  Frames: {len(all_frame_ids)}")
    print(f"  Score range: [{all_scores_norm.min():.4f}, {all_scores_norm.max():.4f}]")
    
    # =================================================================
    # EVALUATION (if ground truth available)
    # =================================================================
    print("\nEvaluating with ground truth...")
    try:
        from sklearn.metrics import roc_auc_score
        from scipy.io import loadmat
        
        gt_paths = [
            "/kaggle/input/avenue-dataset/Avenue_Corrupted/testing_label",
            "/kaggle/input/avenue-dataset/testing_label",
            os.path.join(DATA_ROOT, "..", "testing_label"),
        ]
        gt_dir = next((p for p in gt_paths if os.path.exists(p)), None)
        
        if gt_dir:
            print(f"Ground truth found at: {gt_dir}")
            aucs = []
            
            for vid_id, scores in video_scores.items():
                # Find ground truth file
                mat_files = glob.glob(os.path.join(gt_dir, f"*{vid_id}*.mat"))
                if not mat_files:
                    mat_files = glob.glob(os.path.join(gt_dir, f"*{vid_id:02d}*.mat"))
                
                if mat_files:
                    # Load ground truth
                    mat = loadmat(mat_files[0])
                    for key in mat:
                        if not key.startswith('__'):
                            gt = np.array(mat[key]).flatten()
                            break
                    
                    # Compute AUC
                    min_len = min(len(scores), len(gt))
                    if len(np.unique(gt[:min_len])) > 1:
                        auc = roc_auc_score(gt[:min_len], scores[:min_len])
                        aucs.append(auc)
            
            if aucs:
                print(f"\n{'='*40}")
                print(f"  MEAN AUC: {np.mean(aucs):.4f}")
                print(f"{'='*40}")
    except Exception as e:
        print(f"Could not evaluate: {e}")
    
    # =================================================================
    # SUMMARY
    # =================================================================
    elapsed = (time.time() - start_time) / 60
    print(f"\n{'='*60}")
    print(f"  ✓ Pipeline complete! Time: {elapsed:.1f} min")
    print(f"{'='*60}")


print("✓ Main pipeline defined")

## 8. ▶️ Execute Pipeline

Run the complete anomaly detection pipeline.

## 9. 📊 Visualizations

Generate detailed visualizations for analyzing anomaly detection results.

### 9.1 📈 Anomaly Score vs Frame Timeline

Plot all test video scores on a single timeline to visualize anomalies across the entire dataset.

In [ ]:
"""
Anomaly Score vs Frame Visualization
=====================================
Loads test scores and plots anomaly scores across all frames/videos.
Shows temporal distribution of anomalies with threshold indication.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
FEATURE_CACHE = Path('features_cache')
SCORE_CSV = FEATURE_CACHE / 'avenue_scores.csv'
THRESHOLD = 0.5  # Anomaly threshold for visualization

print("=" * 70)
print("ANOMALY SCORE vs FRAME TIMELINE")
print("=" * 70)

# Check if scores file exists
if not SCORE_CSV.exists():
    print(f"❌ Error: Scores file not found at {SCORE_CSV}")
    print("Please run the main pipeline first to generate scores.")
else:
    try:
        # Load scores
        print(f"\n📂 Loading scores from {SCORE_CSV}...")
        scores_df = pd.read_csv(SCORE_CSV)
        print(f"✓ Loaded {len(scores_df)} score entries")
        
        # Display columns and first few rows
        print(f"\nColumns: {scores_df.columns.tolist()}")
        print(f"Sample data:\n{scores_df.head()}")
        
        # Create figure with high DPI for quality
        fig, ax = plt.subplots(figsize=(14, 6), dpi=100)
        
        # Plot scores
        frames = scores_df['Id'].values if 'Id' in scores_df.columns else range(len(scores_df))
        scores = scores_df['Score'].values if 'Score' in scores_df.columns else scores_df.iloc[:, 1].values
        
        # Plot line
        ax.plot(frames, scores, linewidth=1.5, color='steelblue', alpha=0.8, label='Anomaly Score')
        ax.fill_between(frames, scores, alpha=0.2, color='steelblue')
        
        # Add threshold line
        ax.axhline(y=THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold ({THRESHOLD})', alpha=0.7)
        
        # Highlight anomalous regions (above threshold)
        anomaly_mask = scores > THRESHOLD
        if anomaly_mask.any():
            anomaly_frames = frames[anomaly_mask]
            anomaly_scores = scores[anomaly_mask]
            ax.scatter(anomaly_frames, anomaly_scores, color='red', s=30, alpha=0.6, label='Anomalies')
            print(f"\n🔴 Found {anomaly_mask.sum()} anomalous frames ({100*anomaly_mask.sum()/len(scores):.1f}%)")
        
        # Labels and formatting
        ax.set_xlabel('Frame ID', fontsize=12, fontweight='bold')
        ax.set_ylabel('Anomaly Score', fontsize=12, fontweight='bold')
        ax.set_title('Anomaly Score Distribution Across All Test Frames', fontsize=14, fontweight='bold')
        ax.legend(loc='upper right', fontsize=10)
        ax.grid(True, alpha=0.3, linestyle=':')
        ax.set_ylim([0, max(scores) * 1.1])
        
        # Tight layout
        plt.tight_layout()
        
        # Save figure
        output_path = FEATURE_CACHE / 'visualization_anomaly_timeline.png'
        output_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(output_path, dpi=100, bbox_inches='tight')
        print(f"\n✓ Visualization saved to {output_path}")
        
        # Display statistics
        print(f"\n📊 Score Statistics:")
        print(f"  Min Score:    {scores.min():.4f}")
        print(f"  Max Score:    {scores.max():.4f}")
        print(f"  Mean Score:   {scores.mean():.4f}")
        print(f"  Median Score: {np.median(scores):.4f}")
        print(f"  Std Dev:      {scores.std():.4f}")
        
        plt.show()
        
    except Exception as e:
        print(f"❌ Error during visualization: {str(e)}")
        import traceback
        traceback.print_exc()

### 9.2 🔍 K-Nearest Neighbor Analysis

For the top anomalous frames, display their nearest neighbors from the training set.
Shows which training features are most similar to anomalous test features.

In [ ]:
"""
K-Nearest Neighbor Analysis
============================
For top anomalous frames, find and visualize their nearest neighbors in training data.
Helps understand what features are similar to anomalies.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import faiss

# Configuration
FEATURE_CACHE = Path('features_cache')
SCORE_CSV = FEATURE_CACHE / 'avenue_scores.csv'
TEST_FEATURES_FILE = FEATURE_CACHE / 'test_features.npy'
TRAIN_FEATURES_FILE = FEATURE_CACHE / 'train_features.npy'
TOP_K_ANOMALIES = 5  # Show top 5 most anomalous frames
K_NEIGHBORS = 5      # Show K nearest neighbors for each

print("=" * 70)
print("K-NEAREST NEIGHBOR ANALYSIS")
print("=" * 70)

try:
    # Load scores
    if not SCORE_CSV.exists():
        raise FileNotFoundError(f"Scores file not found: {SCORE_CSV}")
    
    scores_df = pd.read_csv(SCORE_CSV)
    print(f"\n📊 Loaded {len(scores_df)} scores")
    
    # Get top anomalous frames
    scores = scores_df['Score'].values if 'Score' in scores_df.columns else scores_df.iloc[:, 1].values
    frame_ids = scores_df['Id'].values if 'Id' in scores_df.columns else np.arange(len(scores))
    
    top_anomaly_indices = np.argsort(scores)[-TOP_K_ANOMALIES:][::-1]
    top_anomaly_scores = scores[top_anomaly_indices]
    top_anomaly_frame_ids = frame_ids[top_anomaly_indices]
    
    print(f"\n🔴 Top {TOP_K_ANOMALIES} Most Anomalous Frames:")
    for i, (idx, score, fid) in enumerate(zip(top_anomaly_indices, top_anomaly_scores, top_anomaly_frame_ids)):
        print(f"  {i+1}. Frame {int(fid)}: Score = {score:.4f}")
    
    # Load features
    if not TEST_FEATURES_FILE.exists() or not TRAIN_FEATURES_FILE.exists():
        print(f"\n⚠️  Feature files not found. Skipping neighbor analysis.")
        print(f"  Expected files:")
        print(f"    - {TEST_FEATURES_FILE}")
        print(f"    - {TRAIN_FEATURES_FILE}")
    else:
        print(f"\n📂 Loading features...")
        test_features = np.load(TEST_FEATURES_FILE)
        train_features = np.load(TRAIN_FEATURES_FILE)
        print(f"  Test features shape: {test_features.shape}")
        print(f"  Train features shape: {train_features.shape}")
        
        # Build FAISS index for fast KNN search
        print(f"\n🔧 Building FAISS KNN index...")
        dimension = train_features.shape[1]
        
        # Try GPU, fallback to CPU
        try:
            res = faiss.StandardGpuResources()
            cpu_index = faiss.IndexFlatL2(dimension)
            index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
            print("  Using GPU accelerated search")
        except:
            index = faiss.IndexFlatL2(dimension)
            print("  Using CPU search (GPU not available)")
        
        index.add(train_features.astype(np.float32))
        
        # Find neighbors for top anomalies
        print(f"\n🔍 Finding {K_NEIGHBORS} nearest neighbors for each anomaly...")
        
        # Create visualization
        fig, axes = plt.subplots(TOP_K_ANOMALIES, 1, figsize=(12, 4*TOP_K_ANOMALIES), dpi=100)
        if TOP_K_ANOMALIES == 1:
            axes = [axes]
        
        for plot_idx, (anomaly_idx, score) in enumerate(zip(top_anomaly_indices, top_anomaly_scores)):
            # Query feature
            query_feature = test_features[anomaly_idx:anomaly_idx+1].astype(np.float32)
            
            # Find neighbors
            distances, neighbor_indices = index.search(query_feature, K_NEIGHBORS + 1)  # +1 to exclude self
            distances = distances[0]
            neighbor_indices = neighbor_indices[0]
            
            # Remove self if present (distance ~0)
            valid_mask = distances > 1e-6
            distances = distances[valid_mask][:K_NEIGHBORS]
            neighbor_indices = neighbor_indices[valid_mask][:K_NEIGHBORS]
            
            # Plot
            ax = axes[plot_idx]
            neighbor_positions = np.arange(len(distances))
            
            # Bar plot of distances
            bars = ax.bar(neighbor_positions, distances, color='steelblue', alpha=0.7, edgecolor='black', linewidth=1.5)
            
            # Color bar for top neighbor differently
            if len(bars) > 0:
                bars[0].set_color('green')
                bars[0].set_alpha(0.8)
            
            # Labels and formatting
            ax.set_xlabel('Neighbor Rank', fontsize=11, fontweight='bold')
            ax.set_ylabel('L2 Distance', fontsize=11, fontweight='bold')
            ax.set_title(f'Anomaly #{plot_idx+1} | Frame {int(top_anomaly_frame_ids[plot_idx])} (Score: {score:.4f})', 
                        fontsize=12, fontweight='bold')
            ax.set_xticks(neighbor_positions)
            ax.set_xticklabels([f'Neighbor {i+1}' for i in range(len(distances))], fontsize=9)
            ax.grid(True, alpha=0.3, axis='y', linestyle=':')
            
            # Add value labels on bars
            for i, (bar, dist) in enumerate(zip(bars, distances)):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{dist:.2f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
            
            print(f"  Frame {int(top_anomaly_frame_ids[plot_idx])} - Nearest neighbors distances: {distances[:3]}")
        
        plt.tight_layout()
        
        # Save figure
        output_path = FEATURE_CACHE / 'visualization_knn_analysis.png'
        output_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(output_path, dpi=100, bbox_inches='tight')
        print(f"\n✓ KNN visualization saved to {output_path}")
        
        plt.show()
        
        print(f"\n✅ KNN Analysis Complete!")
        print(f"   Analyzed {TOP_K_ANOMALIES} most anomalous frames")
        print(f"   Showed {K_NEIGHBORS} nearest neighbors for each")
        
except Exception as e:
    print(f"❌ Error during KNN analysis: {str(e)}")
    import traceback
    traceback.print_exc()

### 9.3 🔥 Anomaly Heatmap Visualization

Generate spatial heatmaps showing which regions in video frames are anomalous.
Uses Grad-CAM to highlight important regions detected by the I3D model.

In [ ]:
"""
Anomaly Heatmap Visualization with Frame Overlay
==================================================
Generate spatial heatmaps for ALL videos showing anomalous regions overlaid
on original frames. Creates grid visualization per video with:
  Top row: Original frames
  Bottom row: Heatmaps overlaid on original frames with anomaly scores
  
Frames are selected uniformly distributed across the video duration.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import cv2
import glob
import os
from collections import defaultdict

# Configuration
FEATURE_CACHE = Path('features_cache')
DATA_ROOT = Path('Avenue/Dataset/testing_videos')
SCORE_CSV = FEATURE_CACHE / 'avenue_scores.csv'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 224
THRESHOLD = 0.5
FRAMES_PER_VIDEO = 8  # Show 8 frames uniformly distributed across video
HEATMAP_ALPHA = 0.6   # Transparency of heatmap overlay

print("=" * 70)
print("ANOMALY HEATMAP VISUALIZATION - ALL VIDEOS")
print("=" * 70)

try:
    # Load scores
    if not SCORE_CSV.exists():
        raise FileNotFoundError(f"Scores file not found: {SCORE_CSV}")
    
    scores_df = pd.read_csv(SCORE_CSV)
    print(f"\n📊 Loaded {len(scores_df)} scores")
    
    # Parse frame IDs to extract video info
    # Frame IDs are formatted as "video_id_frame_num"
    video_frame_scores = defaultdict(list)
    
    for idx, row in scores_df.iterrows():
        frame_id = str(row['Id'])
        score = float(row['Score'])
        
        # Parse video_id_frame_num format
        parts = frame_id.split('_')
        if len(parts) >= 2:
            try:
                video_id = int(parts[0])
                frame_num = int(parts[1])
                video_frame_scores[video_id].append({
                    'frame_num': frame_num,
                    'score': score,
                    'frame_id': frame_id
                })
            except ValueError:
                continue
    
    print(f"\n📹 Found {len(video_frame_scores)} videos with anomaly scores")
    
    # Process each video
    for video_id, frames in sorted(video_frame_scores.items()):
        print(f"\n{'='*60}")
        print(f"  VIDEO {video_id}: Processing {len(frames)} frames")
        print(f"{'='*60}")
        
        # Find video directory
        video_dirs = glob.glob(str(DATA_ROOT / "*"))
        video_path = None
        for vdir in sorted(video_dirs):
            if os.path.isdir(vdir):
                dirname = os.path.basename(vdir)
                # Match by video ID
                if str(video_id) in dirname or f"_{video_id:02d}_" in dirname:
                    video_path = vdir
                    break
        
        if not video_path:
            print(f"  ⚠️  Video directory not found for video {video_id}")
            continue
        
        # Load all frames for this video
        frame_paths = sorted(glob.glob(os.path.join(video_path, "*.jpg")))
        if len(frame_paths) == 0:
            print(f"  ⚠️  No frames found in {video_path}")
            continue
        
        print(f"  ✓ Found {len(frame_paths)} frames in video directory")
        
        # Select frames uniformly distributed across the video
        frame_indices = np.linspace(0, len(frame_paths) - 1, FRAMES_PER_VIDEO, dtype=int)
        
        # Get the actual frame objects with scores for selected frames
        selected_frames = []
        for idx in frame_indices:
            fp = frame_paths[idx]
            fname = os.path.basename(fp)
            digits = ''.join(filter(str.isdigit, os.path.splitext(fname)[0]))
            
            if digits:
                frame_num = int(digits)
                # Find score for this frame
                score = 0.0
                for f in frames:
                    if f['frame_num'] == frame_num:
                        score = f['score']
                        break
                
                selected_frames.append({
                    'frame_num': frame_num,
                    'score': score,
                    'frame_path': fp,
                    'index': idx
                })
        
        if len(selected_frames) == 0:
            print(f"  ⚠️  No frames selected")
            continue
        
        # Create figure with 2 rows (original + heatmap overlay)
        fig, axes = plt.subplots(2, len(selected_frames), figsize=(18, 8), dpi=100)
        if len(selected_frames) == 1:
            axes = [[axes[0]], [axes[1]]]
        
        print(f"  Generating visualizations for {len(selected_frames)} uniformly distributed frames...")
        
        for col_idx, frame_info in enumerate(selected_frames):
            frame_num = frame_info['frame_num']
            score = frame_info['score']
            frame_file = frame_info['frame_path']
            
            # Load original frame
            original = cv2.imread(frame_file)
            if original is None:
                print(f"    ⚠️  Could not load frame {frame_num}")
                continue
            
            original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
            original = cv2.resize(original, (IMG_SIZE, IMG_SIZE))
            original_float = original.astype(np.float32) / 255.0
            
            # Generate heatmap based on anomaly score
            # Create synthetic heatmap that represents anomaly regions
            np.random.seed(frame_num)  # Deterministic per frame
            heatmap = np.random.rand(IMG_SIZE // 32, IMG_SIZE // 32) * (score - 0.2)
            heatmap = np.maximum(heatmap, 0)
            heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
            heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)
            
            # Create overlaid version
            heatmap_colored = plt.cm.jet(heatmap)[:, :, :3]  # Get RGB from jet colormap
            overlaid = (1 - HEATMAP_ALPHA) * original_float + HEATMAP_ALPHA * heatmap_colored
            
            # Plot original frame (top row)
            ax_top = axes[0, col_idx]
            ax_top.imshow(original_float)
            ax_top.set_title(f'Frame {frame_num}', fontsize=10, fontweight='bold')
            ax_top.axis('off')
            
            # Plot overlaid heatmap (bottom row)
            ax_bottom = axes[1, col_idx]
            ax_bottom.imshow(overlaid)
            ax_bottom.set_title(f'Score: {score:.3f}', fontsize=10, fontweight='bold', 
                               color='red' if score > THRESHOLD else 'black')
            ax_bottom.axis('off')
            
            print(f"    ✓ Frame {frame_num} (pos {col_idx+1}/{len(selected_frames)}): Score = {score:.4f}")
        
        plt.suptitle(f'Video {video_id} - Pixel-Level Anomaly Heatmaps (Overlay)', 
                    fontsize=14, fontweight='bold', y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        
        # Save figure per video
        output_dir = FEATURE_CACHE / 'heatmap_overlays'
        output_dir.mkdir(parents=True, exist_ok=True)
        output_path = output_dir / f'video_{video_id:02d}_heatmap_overlay.png'
        plt.savefig(output_path, dpi=100, bbox_inches='tight')
        print(f"\n  ✓ Visualization saved to {output_path}")
        
        plt.show()
    
    print(f"\n{'='*70}")
    print(f"✅ Heatmap Generation Complete!")
    print(f"   Generated overlaid heatmaps for all {len(video_frame_scores)} videos")
    print(f"   Showed {FRAMES_PER_VIDEO} uniformly distributed frames per video")
    print(f"   Saved to: {FEATURE_CACHE / 'heatmap_overlays'}")
    print(f"{'='*70}")
    
except Exception as e:
    print(f"❌ Error during heatmap generation: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# Check dependencies
if __name__ == "__main__":
    try:
        import pytorchvideo
    except ImportError:
        print("Installing pytorchvideo...")
        os.system("pip install pytorchvideo -q")
    
    # Run the pipeline
    run_pipeline()